In [4]:
import psycopg2
import pandas as pd
from config import params

#performing connection to PostgreSQL database, calling connect() method and receaving Connection object that is stored into conn variable
conn = psycopg2.connect(**params)
#using connection object and cursor() method creating cursor object to be able to execute PostgreSQL queries from Python
cur = conn.cursor()

#execute select query using execute() method
cur.execute("""SELECT * FROM real_flight WHERE cancelled = 0 AND diverted = 0""")
#retreaving all rows from table real_flight
rows = cur.fetchall()

#closing database connection
cur.close()
conn.close()

#creating dataframe with the retrieved data
df = pd.DataFrame(rows, columns=[desc.name for desc in cur.description])

In [6]:
#dropping all rows that contain "NaN" in either arr_del15 or dep_del15 
df.dropna(subset=['arr_del15','dep_del15' ], how='all', inplace=True)


In [68]:
#creating new column with df dataframe, True value if there is a delay on either arrival or departure
df['delayed'] = (df.arr_del15==1) | (df.dep_del15==1)
df['delayed'] = df.delayed.astype(int)

In [46]:
#creating new datafarme with each airline code and a ratio of delays for each of them
del_airline = pd.DataFrame(df.groupby('op_unique_carrier')['delayed'].mean())
#resetting to default 0-.. indexing, op_unique_carrier is set as an attribute
del_airline.reset_index(level=0,inplace=True)

In [47]:
#sorting the delay ratio from highest to lowest, resetting the index according to the new order
del_airline = del_airline.sort_values(by='delayed', ascending=False).reset_index(drop=True)

In [48]:
#saving new data frame as a csv file
del_airline.to_csv('delayed_airlines.csv')

In [36]:
#creating new data frame that would store ratios of delays for each airport
del_airp = pd.DataFrame(df.groupby('origin_airport_id')['delayed'].mean())
del_airp.reset_index(level=0,inplace=True)

In [38]:
#sorting delays ratios from highest to lowest, resetting indexing
del_airp = del_airp.sort_values(by='delayed', ascending=False).reset_index(drop=True)


In [39]:
#saving del_airp datafarme as csv
del_airp.to_csv('delayed_airports.csv')

Challenge


In [45]:
#loading the csv file into jd_ratings dataframe
jd_ratings = pd.read_csv('jd_ratings.csv')

,code,name,rating
0,WN,SouthWest Airlines,839
1,B6,JetBlue Airways,833
2,AS,Alaska Airlines,828
3,DL,Delta Air Lines,820
4,WS,WestJet,787
5,AA,American Airlines,779
6,AX,Air Canada,777
7,NK,Spirit Airlines,773
8,F9,Frontier Airlines,746


In [51]:
#renaming a column 'op_unique_carrier' to 'code' to avoid columns with duplicate data in the next step
del_airline.rename(columns={'op_unique_carrier':'code'},inplace=True)

In [69]:
#merging two dataframes to align each airline with its ratio of delays and rating
result = pd.merge(del_airline,jd_ratings,on='code')


In [66]:
# importing linear regression model from Scikit-learn to generate R^2 value
from sklearn.linear_model import LinearRegression
#Reshaping data either using double []
X = result[['delayed']]
y = result['rating']

model = LinearRegression()
model.fit(X, y)

r_2 = model.score(X, y)
print(r_2)

0.00019317587980038908


The R^2 < 0.5, it's insignificant, meaning ratio of delays might not predict customer satisfaction.